In [ ]:
# 03_ukf_demo.ipynb  ── run cell ─────────────────────────────────────────────
%matplotlib inline
import json, numpy as np, pandas as pd, matplotlib.pyplot as plt

# 1) Load pre-processed measurements ----------------------------------------
df         = pd.read_parquet("GPS_clean.parquet")
meas_cols  = json.load(open("meas_cols.json"))
meas       = df[meas_cols].values.astype("float64")        # (N, 6)
epochs     = df["time"].values.astype("datetime64[s]")

# 2) UKF + Orekit propagator -----------------------------------------------
from utils import OrbitPropagator      # auto-registers orekit-data
from aukf  import UnscentedKalman

prop = OrbitPropagator()               # ⭐ NEW – create the propagator
ukf  = UnscentedKalman(prop, meas_cols,    # ⭐ pass meas_cols, not cols
                       q0=1e-2, r0=25.0,
                       adaptive="iae")     # Innovation-Based Adaptive Est.

ukf.init_from_measurement(0.0, meas[0])
prev_t = epochs[0]

hist = [ukf.x.copy()]
for t, z in zip(epochs[1:], meas[1:]):
    dt = (t - prev_t).astype(int)
    ukf.step(dt, z)
    hist.append(ukf.x.copy())
    prev_t = t

hist = np.asarray(hist)

# 3) Ground-track quick-look -----------------------------------------------
plt.figure(figsize=(6,6))
plt.plot(hist[:,0]/1e3, hist[:,1]/1e3, lw=0.4)
plt.gca().set_aspect("equal")
plt.xlabel("X [km]"); plt.ylabel("Y [km]")
plt.title("UKF ground-track (ECEF)")
plt.tight_layout()
plt.show()
